Данный блокнот содержит все имеющиеся сведения по работе над датасетом первых воспоминаний.

Черновой план дальнейшей работы:
- Инструмент для представления исходных данных и обработанных в верстке удобной для публикации в pdf (LaTeX), в виде статьи на Medium
- Описать подробнее в MarkDown файле - две ссылки - на источник и сам опрос, перечислить какие поля содержит датасет
- Сделать переводную версию
- Объединить функции для удобного обновления данных при внесении исправлении в таблицу
- Преобразование в pandas строковых значений чисел в числовые
- Инструмент перечисления номеров воспоминаний, в которых не заполнено переданное поле
- Инструмент оценки полноты датафрейма с рекомендациями работы над датафреймом
- Инструменты визуализации статистических распределений
- Использовать https://github.com/natasha/natasha и pymorphy
- pymorphy: какое время используется у глаголов, частотное распределение слов, приведенных к нормальной форме, разбивать составные воспоминания по отступ в виде одной строки или специальному символу. Отношение между частями речи. Что преобладает в коротких воспоминаниях? Анализ по парам младший/старший, мама/пара, бабушка/дедушка.

# Считывание данных из Google Sheet

Набор данных оформлен [в виде электронной таблицы на Google Drive](https://docs.google.com/spreadsheets/d/1KSirtO9hZSmVst--GiqsBPYk6hX-xOCI-SolgiafjcI/edit?usp=sharing). Доступ к Google Sheets API осуществляется в соответствии с [документацией API для Python](https://developers.google.com/sheets/api/quickstart/python).

Импортируем данные учетной записи и преобразуем в объект Pandas:

In [1]:
import json
import os.path
import pickle       # для хранения токенов
import pandas as pd
    
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SPREADSHEET_ID = '1KSirtO9hZSmVst--GiqsBPYk6hX-xOCI-SolgiafjcI' # id гугл-таблицы
RANGE_NAME = 'Memories' # Забираем лист целиком диапазон

In [2]:
def get_data():
    """Авторизует и забирает данные из Google SpreadSheet"""
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
    values = result.get('values', [])

    if not values:
        print('No data found.')
    
    return values

data = get_data()

## Обработка данных в Pandas

Создаем датафрейм. Удаляем те строки, что не относятся к датасету (комментарии в конце файла)

In [3]:
df = pd.DataFrame.from_records(data)
headers = df.iloc[0]  # Названия столбцов содержатся в нулевой строке таблицы
df = pd.DataFrame(df.values[1:], columns=headers)
df = df.set_index('№') # Номер строки соответствует номеру воспоминания

Все столбцы датасета:

In [4]:
for column in df.columns:
    print(column)
    
not_quant_cols = ['Translation', 'Translation footnote', 'Комментарий']

Имя
Name
Воспоминание
Translation
Translation footnote
Пол
Кол-во
Возраст
Ориентир возраста
Лица
Предметы
Цвета
Звуки
Запахи, вкус
Другие ощущения
Сущности
Пространство
Время дня
Время года
Поведение
Комментарий


In [5]:
def col_empty_cells(col_name:str):
    """Находит пустые и NoneType ячейки столбца с именем col_name"""
    none_cells = df[col_name][~df[col_name].notna()].index
    empty_cells = df[col_name][df[col_name]==''].index
    ids = sorted(list(set(none_cells) | set(empty_cells)))
    return ids

col_name = 'Кол-во'
ids = col_empty_cells(col_name)
print('{0} записи столбца "{1}" пусты: {2}'.format(len(ids), col_name, list(ids)))

41 записи столбца "Кол-во" пусты: ['217', '222', '226', '232', '238', '240', '241', '242', '243', '244', '245', '246', '247', '248', '249', '250', '252', '253', '254', '256', '257', '258', '260', '263', '265', '266', '293', '323', '343', '356', '385', '407', '423', '426', '476', '512', '515', '537', '554', '599', '614']


In [6]:
def percent():
    empty_count = 0
    for col_name in df.columns:
        if col_name not in not_quant_cols:
            number = len(col_empty_cells(col_name))
            print(col_name, number)
            empty_count += number
    p = 100*(1-empty_count/(df.size-len(df)*len(not_quant_cols)))
    print("Доля частично или полно обработанных ячеек составляет {0:.1f}%".format(p))

In [7]:
percent()

Имя 0
Name 2
Воспоминание 1
Пол 0
Кол-во 41
Возраст 88
Ориентир возраста 379
Лица 200
Предметы 374
Цвета 357
Звуки 377
Запахи, вкус 389
Другие ощущения 388
Сущности 387
Пространство 378
Время дня 391
Время года 370
Поведение 387
Доля частично или полно обработанных ячеек составляет 61.2%
